In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from childes_mi.utils.paths import DATA_DIR, FIGURE_DIR
from childes_mi.utils.general import flatten,save_fig

In [3]:
from childes_mi.information_theory import model_fitting as mf

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
MI_DF = pd.read_pickle(DATA_DIR/'mi/phonbank_mi.pickle')

In [6]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_phones,n_unique_phones
0,"[10.080288260013138, 9.916455549165027, 9.7424...","[0.010088692845104597, 0.009966629345003522, 0...","[9.540319164934521, 9.545878218535728, 9.53434...","[0.009889103513194912, 0.009903386166469113, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,60390,3259
1,"[11.018700216584904, 10.737645462157055, 10.56...","[0.005213049628772987, 0.005149803969908309, 0...","[10.354873873252643, 10.35376084240053, 10.353...","[0.00508955241492165, 0.005096158572864715, 0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1.0,1.5,312914,5787
2,"[11.052948976446858, 10.656466560369868, 10.48...","[0.0044666661484878325, 0.0043938556743630605,...","[10.335267645293385, 10.33318150766475, 10.333...","[0.004350729568225813, 0.004350829245781982, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1.5,2,457938,5769


In [7]:
MI_DF = MI_DF.assign(**{i:np.nan for i in ['exp_results', 'pow_results', 'concat_results',
     'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
     'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak']})
MI_DF['curvature'] = MI_DF['curvature'].astype(object)

In [8]:
n = 100 # max distance for computation
for idx, row in tqdm(MI_DF.iterrows(), total=len(MI_DF)):
    # get signal
    sig = np.array(row.MI-row.shuff_MI)
    distances = row.distances
    sig = sig
    
    # fit models
    results_power, results_exp, results_pow_exp, best_fit_model = mf.fit_models(distances, sig)
    
    # get fit results
    R2_exp, R2_concat, R2_power, AICc_exp, \
        AICc_pow, AICc_concat = mf.fit_results(sig, distances, 
                                              results_exp, results_power,
                                              results_pow_exp)
    
    
    # get model y
    distances_mod = np.logspace(0,np.log10(n), base=10, num=1000)
    if best_fit_model == 'pow_exp':
        y_model = mf.get_y(mf.pow_exp_decay, results_pow_exp, distances_mod)
    elif best_fit_model == 'exp':
        y_model = mf.get_y(mf.exp_decay, results_exp, distances_mod)
    elif best_fit_model == 'pow':
        y_model = mf.get_y(mf.powerlaw_decay, results_power, distances_mod)
    
    # get curvature of model_y
    curvature_model = mf.curvature(np.log(y_model))
    
    # if the best fit model is pow_exp, then grab the min peak
    if best_fit_model == 'pow_exp':
        # get peaks of curvature
        peaks = np.where((
            (curvature_model[:-1] < curvature_model[1:])[1:] & (curvature_model[1:] < curvature_model[:-1])[:-1]
        ))
        min_peak = peaks[0][0]
    else:
        min_peak = np.nan

    # get save model fit results to MI_DF
    MI_DF.loc[idx, np.array(['exp_results', 'pow_results', 'concat_results',
                         'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
                         'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'])] = [
        results_exp, results_power, results_pow_exp,
        R2_exp, R2_concat, R2_power, AICc_exp,
        AICc_concat, AICc_pow, best_fit_model,
        curvature_model, min_peak
    ]

    print(row.age_cohort_low, row.age_cohort_high, best_fit_model)

0.5 1 pow_exp
1.0 1.5 pow_exp
1.5 2 pow_exp
2.0 2.5 pow_exp
2.5 3 pow_exp
3.0 20 pow_exp



In [10]:
MI_DF

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_phones,n_unique_phones,exp_results,...,concat_results,R2_exp,R2_concat,R2_power,AICc_exp,AICc_concat,AICc_power,bestfitmodel,curvature,min_peak
0,"[10.080288260013138, 9.916455549165027, 9.7424...","[0.010088692845104597, 0.009966629345003522, 0...","[9.540319164934521, 9.545878218535728, 9.53434...","[0.009889103513194912, 0.009903386166469113, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,60390,3259,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.983485,0.993614,0.989926,-573.660113,-664.289186,-623.093092,pow_exp,"[-4.7259173188471115e-07, -7.152342473020701e-...",259.0
1,"[11.018700216584904, 10.737645462157055, 10.56...","[0.005213049628772987, 0.005149803969908309, 0...","[10.354873873252643, 10.35376084240053, 10.353...","[0.00508955241492165, 0.005096158572864715, 0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1.0,1.5,312914,5787,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.980861,0.998117,0.995941,-516.601598,-744.125823,-671.671083,pow_exp,"[-6.398269681238529e-07, -9.676437685993425e-0...",247.0
2,"[11.052948976446858, 10.656466560369868, 10.48...","[0.0044666661484878325, 0.0043938556743630605,...","[10.335267645293385, 10.33318150766475, 10.333...","[0.004350729568225813, 0.004350829245781982, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1.5,2,457938,5769,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.971529,0.998164,0.996968,-436.462823,-706.207524,-660.428461,pow_exp,"[-3.526032069378871e-06, -5.3052760008367026e-...",169.0
3,"[11.117589847657872, 10.672792563469578, 10.48...","[0.0040212095683027855, 0.003940573995629109, ...","[10.339949506377152, 10.339290626343217, 10.33...","[0.0038946673782298625, 0.003895116542366042, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2.0,2.5,545062,5448,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.959973,0.996074,0.993765,-385.044117,-612.847682,-570.975906,pow_exp,"[-6.582241601689192e-06, -9.892304726140322e-0...",163.0
4,"[11.121574094494914, 10.664656344849487, 10.46...","[0.0036472645805226984, 0.003559756005532006, ...","[10.326966868664798, 10.327498323812126, 10.32...","[0.0035076360836199492, 0.0035094205130675815,...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2.5,3,552929,4817,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.957094,0.990644,0.988273,-366.593177,-514.503850,-496.304608,pow_exp,"[-6.551685972732687e-06, -9.847877058996883e-0...",178.0
5,"[10.355129199937373, 9.874298956040494, 9.6434...","[0.00503053268630887, 0.00490644511432297, 0.0...","[9.476918105199086, 9.476465553083116, 9.47586...","[0.004822528611486606, 0.0048213731214393875, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3.0,20,310065,2574,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.946653,0.986017,0.982729,-339.366979,-468.876358,-452.142734,pow_exp,"[-6.801322960555882e-06, -1.0220997117116343e-...",193.0


In [11]:
labels = ['6-12 months', '12-18 months', '18-24 months', '24-30 months', '30-36 months', '3-4 years']

### Calculate R2

In [12]:
R2 = MI_DF[['R2_exp', 'R2_concat', 'R2_power']]
R2.columns = ['exp', 'combined', 'power-law']
R2.index = labels
R2 = R2.T
R2

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-4 years
exp,0.983485,0.980861,0.971529,0.959973,0.957094,0.946653
combined,0.993614,0.998117,0.998164,0.996074,0.990644,0.986017
power-law,0.989926,0.995941,0.996968,0.993765,0.988273,0.982729


### AICc

In [13]:
AICcs = MI_DF[['AICc_exp', 'AICc_concat', 'AICc_power']]
AICcs.columns = ['exp', 'combined.', 'power-law']
AICcs.index = labels
AICcs = AICcs.T
AICcs

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-4 years
exp,-573.660113,-516.601598,-436.462823,-385.044117,-366.593177,-339.366979
combined.,-664.289186,-744.125823,-706.207524,-612.847682,-514.503850,-468.876358
power-law,-623.093092,-671.671083,-660.428461,-570.975906,-496.304608,-452.142734


#### $\Delta$AIC

In [14]:
delta_AICcs = AICcs.T - np.repeat(np.min(AICcs.T.values, axis=1),3).reshape(len(labels),3)
delta_AICcs = delta_AICcs.T
delta_AICcs

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-4 years
exp,90.629074,227.524225,269.744701,227.803565,147.910672,129.509378
combined.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
power-law,41.196094,72.454740,45.779063,41.871776,18.199242,16.733624


### relative likelihood

In [15]:
relative_likelihoods = mf.relative_likelihood(delta_AICcs)
relative_likelihoods

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-4 years
exp,2.090002e-20,3.924119e-50,2.664910e-59,3.412593e-50,7.613873e-33,7.540425e-29
combined.,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
power-law,1.133396e-09,1.847792e-16,1.146048e-10,8.084616e-10,1.117082e-04,2.324554e-04


### relative probability of models

In [16]:
prob_models = mf.Prob_model_Given_data_and_models(relative_likelihoods)
prob_models

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-4 years
exp,2.090002e-20,3.924119e-50,2.664910e-59,3.412593e-50,7.613022e-33,7.538672e-29
combined.,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.998883e-01,9.997676e-01
power-law,1.133396e-09,1.847792e-16,1.146048e-10,8.084616e-10,1.116957e-04,2.324014e-04


#### Join tables

In [17]:
AICcs['superlabel'] = 'AICc'
relative_likelihoods['superlabel'] = 'Relative likelihood'
R2['superlabel'] = '$r^2$'
prob_models['superlabel'] = 'Relative probability'

In [18]:
results_table = pd.concat([
    AICcs,
    R2,
    relative_likelihoods,
    prob_models
]).round(3).replace(0, '<0.001').replace(1, '>0.999')
results_table[''] = results_table.index
results_table.set_index(['superlabel', ''], inplace=True)
results_table

6-12 months 12-18 months 18-24 months  \
superlabel                                                             
AICc                 exp           -573.66     -516.602     -436.463   
                     combined.    -664.289     -744.126     -706.208   
                     power-law    -623.093     -671.671     -660.428   
$r^2$                exp             0.983        0.981        0.972   
                     combined        0.994        0.998        0.998   
                     power-law        0.99        0.996        0.997   
Relative likelihood  exp            <0.001       <0.001       <0.001   
                     combined.      >0.999       >0.999       >0.999   
                     power-law      <0.001       <0.001       <0.001   
Relative probability exp            <0.001       <0.001       <0.001   
                     combined.      >0.999       >0.999       >0.999   
                     power-law      <0.001       <0.001       <0.001   

                               24-30 months 30-36 months 3-4 years  
superlabel                                                          
AICc                 exp           -385.044     -366.593  -339.367  
                     combined.     -612.848     -514.504  -468.876  
                     power-law     -570.976     -496.305  -452.143  
$r^2$                exp               0.96        0.957     0.947  
                     combined         0.996        0.991     0.986  
                     power-law        0.994        0.988     0.983  
Relative likelihood  exp             <0.001       <0.001    <0.001  
                     combined.       >0.999       >0.999    >0.999  
                     power-law       <0.001       <0.001    <0.001  
Relative probability exp             <0.001       <0.001    <0.001  
                     combined.       >0.999       >0.999    >0.999  
                     power-law       <0.001       <0.001    <0.001

In [19]:
results_string = results_table.to_latex(bold_rows=True, escape=False)\
      .replace('>', '$>$')\
      .replace('<', '$<$')\
      .replace('superlabel', '')\
     .replace('\n\\textbf', '\n\midrule\n\\textbf')
print(results_string)

\begin{tabular}{llllllll}
\toprule
                     &           & 6-12 months & 12-18 months & 18-24 months & 24-30 months & 30-36 months & 3-4 years \\
\midrule
\textbf{} & {} &             &              &              &              &              &           \\
\midrule
\midrule
\textbf{AICc} & \textbf{exp} &     -573.66 &     -516.602 &     -436.463 &     -385.044 &     -366.593 &  -339.367 \\
                     & \textbf{combined.} &    -664.289 &     -744.126 &     -706.208 &     -612.848 &     -514.504 &  -468.876 \\
                     & \textbf{power-law} &    -623.093 &     -671.671 &     -660.428 &     -570.976 &     -496.305 &  -452.143 \\
\midrule
\textbf{$r^2$} & \textbf{exp} &       0.983 &        0.981 &        0.972 &         0.96 &        0.957 &     0.947 \\
                     & \textbf{combined} &       0.994 &        0.998 &        0.998 &        0.996 &        0.991 &     0.986 \\
                     & \textbf{power-law} &        0.99 &        0.996 &  